# *VBN - May 2024*
<img src="https://brainmapportal-live-4cc80a57cd6e400d854-f7fdcae.divio-media.net/filer_public_thumbnails/filer_public/6b/da/6bdafa89-61e1-40f8-a517-3186a05f9734/image_sets_and_training_trajectories_diagram.png__1756x1045_q90_subsampling-2.png" width="380" />

## Run the update/reset shortcut on the desktop before each experiment
***
***
# **Without mouse on stage**

In [ ]:
import contextlib
import re
import time

import np_config
import np_jobs
import np_logging
import np_services
import npc_shields
import np_session
import np_workflows
from np_workflows import npxc

from np_services.resources.zro import ZroError 
import contextlib

import vbn_2024 as vbn

logger = np_logging.getLogger()

np_workflows.elapsed_time_widget()

***
## Quiet mode
**on**  [*default*]
- error details are hidden
- regular messages displayed (log level = INFO)

**off**
- full error details (with traceback)
- extra messages displayed (log level = DEBUG)

In [ ]:
np_workflows.quiet_mode_widget()

***
## Launch apps via RSC
[optional]

In [ ]:
with contextlib.suppress(Exception):
    np_services.start_rsc_apps()

***
## Select mouse and user

In [ ]:
user, mouse = np_workflows.user_and_mouse_widget()

***
## Check MTrain and select workflow
Re-run cell this cell if mouse ID is changed

In [ ]:
np_workflows.mtrain_widget(mouse)

---
## Select workflow

In [ ]:
vbn.workflow_select_widget

***
## Generate new session
Check mouse ID and session are correct: this cell will lock them in!

In [ ]:
selected_workflow = vbn.Workflow[vbn.workflow_select_widget.value]
experiment: np_workflows.PipelineExperiment = vbn.new_experiment(mouse, user, selected_workflow)
session: np_session.PipelineSession = experiment.session
platform_json: np_session.PlatformJson = experiment.session.platform_json

platform_json.workflow_start_time = npxc.now()
hab: bool = isinstance(experiment, vbn.Hab)
pretest: bool = (not hab) and experiment.workflow == vbn.Workflow.PRETEST
ephys: bool = (not hab) and experiment.workflow == vbn.Workflow.EPHYS

***
## Checks before starting

### Check ephys day is correct
change stage in MTrain widget above if necessary then re-run this cell to confirm

In [ ]:
if not hab:
    print(f"ephys day: {experiment.ephys_day}")

In [ ]:
np_workflows.check_hardware_widget()

In [ ]:
if not hab:
    np_workflows.check_openephys_widget()

***
## Setup, test, reset all components
*This cell must not be skipped!*

In [ ]:
with contextlib.suppress(ZroError):
    experiment.initialize_and_test_services()

---
## MouseDirector: extend lick spout and set position for mouse
- so it doesn't fly out to an unknown position when the mouse is on the stage

***
## Dip probes

In [ ]:
if not hab:
    print(np_workflows.dye_info_widget.__doc__)
    np_workflows.dye_info_widget(session)

## Photodoc of probes in dye

In [ ]:
if not hab:
    print(str(session) + '_surface-image1-left')

## Probe depths in dye

In [ ]:
if not hab:
    np_workflows.probe_depth_widget(session)

***
***
# **With mouse on stage**
## Before lowering cartridge

In [ ]:
experiment.log('Mouse on stage')
platform_json.HeadFrameEntryTime = npxc.now()
np_workflows.wheel_height_widget(session)
np_workflows.check_mouse_widget

***
## When cartridge is lowered

### Set zoom to 4.0 for photodocs of brain
- focus on the brain surface

## Photodoc of brain (tap probes if hab Day1 or Day2)

In [ ]:
platform_json.CartridgeLowerTime = npxc.now()
print(str(session) + '_surface-image2-left')

***
## ISI map

In [ ]:
np_workflows.isi_widget(mouse.lims)

***
## Probe insertion

- probes will go in the same holes as previous days
- **try to shift each probe within its hole to insert it in a different bit of
  brain**
  - any photodocs from previous days are shown below:

In [ ]:
if ephys:
    display(experiment.get_previous_photodocs_widget())

## Record holes used in implant
if not inserted, move slider to left (value = `None`)

In [ ]:
if not hab:
    widget = npc_shields.get_insertion_widget(
        save_paths=session.npexp_path / 'insertions.json',
        session=session.folder,
        shield_name='2002',  
        experiment_day=experiment.ephys_day,
    )
    
    display(widget)

***
## Photodoc before advancing probes

In [ ]:
if not hab:
    print(str(session) + '_surface-image3-left')

---
## Start probes advancing in NewScale and log time

In [ ]:
if not hab:
    platform_json.ProbeInsertionStartTime = npxc.now()


## Extra advance & retract each probe
- use NewScale GUI to advance an extra 100 $\mu m$ at 200 $\mu m/s$, then reverse 100 $\mu m$ at the same rate

***
## Settle timer & insertion notes & turn on laser

- run both cells now: settle timer will start

- fill out probe notes while waiting

- press Save once

- notes are saved when the timer finishes (button will turn green to confirm)

### *also turn on laser while waiting...*

In [ ]:
if not hab:
    np_workflows.insertion_notes_widget(session)

In [ ]:
if not hab:
    experiment.set_dark_desktop_on_stim()
    experiment.log('settle timer started')
    np_workflows.print_countdown_timer(minutes=.1 if pretest else 30)
    experiment.log('settle timer finished')

***
## Photodoc after probes settled, before experiment

In [ ]:
if not hab:
    print(str(session) + '_surface-image4-left')

In [ ]:
np_workflows.pre_stim_check_widget()

---
### *Before recording: make sure sorting queue is not running!*
-  `run_sorting.exe`
- window may be minimized

--- 
## Flush line before starting experiment 
### Set mouse offset in MouseDirector
- ideally the line should be flushed and checked just before the lick spout is extended, but that won't be possible if all the stimulus cells are queued up together, so do it now

***
## Start devices recording

In [ ]:
experiment.initialize_and_test_services()   # re-do this cell from earlier, right before recording

In [ ]:
experiment.start_recording()

---
## Without lickspout

In [ ]:
if not hab:
    np_services.MouseDirector.get_proxy().retract_lick_spout()
    experiment.run_script('mapping')

---
## With lickspout

In [ ]:
np_services.MouseDirector.get_proxy().extend_lick_spout()


### Give mouse a little water by flushing the line very briefly. Make sure the lick spout looks good and the mouse consumes the water.

In [ ]:
np_services.MouseDirector.get_proxy().extend_lick_spout() # added here just in case any previous extensions were skipped
experiment.run_script('behavior')

---
## Without lickspout

In [ ]:
np_services.MouseDirector.get_proxy().retract_lick_spout()
experiment.run_script('replay')

In [ ]:
if not hab:
    experiment.run_script('optotagging')

***
## Stop recording

In [ ]:
with contextlib.suppress(ZroError):
    experiment.stop_recording_after_stim_finished()

In [ ]:
np_services.MouseDirector.get_proxy().retract_lick_spout()
experiment.reset_desktop_on_stim()

***
## Before removing probes

In [ ]:
if not hab:
    print(str(session) + '_surface-image5-left')

***
## After fully retracting probes

In [ ]:
if not hab:    
    print(str(session) + '_surface-image6-left')

***
## After raising cartridge

In [ ]:
platform_json.HeadFrameExitTime = npxc.now()

np_workflows.finishing_checks_widget()

## Finalize

In [ ]:
platform_json.workflow_complete_time = npxc.now()

experiment.finalize_services(*experiment.recorders, *experiment.stims)
experiment.validate_services(*experiment.recorders, *experiment.stims)

## Copy data

In [ ]:
experiment.copy_data_files()

# Add to post-experiment pipeline

**hab**
- add session to QC queue

**ephys**
- add session to np-exp upload queue, specifying this rig's Acq as `hostname`
    - ensures checksum-validated copy of ephys data on np-exp
    - then adds session to spike-sorting queue
    - then adds session to QC queue

    
    #### run *"process sorting queue .exe"* on Acq desktop

In [ ]:
if hab:
    np_jobs.PipelineQCQueue().add_or_update(session, priority=99)
else:
    np_jobs.PipelineNpexpUploadQueue().add_or_update(session, hostname=np_config.Rig().Acq, priority=99)